In [ ]:
#import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import codecademylib3

#load data
forests = pd.read_csv('forests.csv')
print(forests.head())
#check multicollinearity with a heatmap
corr_matrix = forests.corr()
sns.heatmap(corr_matrix,cmap='PuOr',
annot=True, annot_kws={"size":7})
plt.show()
plt.clf()
# high collinear variables: BUI~DMC, BUI~DC, ISI~FWI (find the dark purple, correlations > 0.9)

#plot humidity vs temperature
#sns.regplot(x=forests["temp"],y=forests["humid"])
sns.lmplot(x="temp",y="humid", data=forests, fit_reg=True, hue="region")
plt.show()
plt.clf()
# the relationship between humidity and temperature appear same for locations in Bejaia and Sidi Bel-abbes. 

#model predicting humidity
modelH = sm.OLS.from_formula('humid~temp+region', data=forests).fit()
print(modelH.params)
print(modelH.params[0])
#modelH.summary()--same with previous

#equations
# full:
# humid = 142.6 - 2.4*temp - 7.2*region
# Bejaia:
# humid = 142.6 - 2.4*temp - 7.2*0
# Sidi:
# humid = 142.6 - 2.4*temp - 7.2*1
#       = 135.3 - 2.4*temp

#interpretations
## Coefficient on temp:
# it represents humid will decrease 2.4 due to a change of one unit in the temperature hold everything else constant
# Intercept:
## For Bejaia equation:
# It indicates that a temperature of zero degrees Celsius is associated with an average relative humidity of 142.6%.
## For Sidi Bel-abbes equation:
# The intercept indicates that a temperature of zero degrees Celsius is associated with an average relative humidity of 135.3%. (it doesn't make sense in the reality but this is how we understand the linear regression model)

#plot regression lines
sns.lmplot(x='temp',y='humid',data=forests,fit_reg=False, hue="region")
plt.plot(forests.temp, modelH.params[0]+modelH.params[1]*0+modelH.params[2]*forests.temp, color='blue',linewidth=3, label='Bejaia')
plt.plot(forests.temp, modelH.params[0]+modelH.params[1]*1+modelH.params[2]*forests.temp, color='red',linewidth=3, label='Bel-abbes')
plt.show()
plt.clf()
#plot FFMC vs temperature
sns.lmplot(x="temp",y="FFMC",data=forests,hue='fire',fit_reg=False)
plt.show()
plt.clf()
# the groups seem to have different slopes
#model predicting FFMC with interaction
modelF=sm.OLS.from_formula('FFMC~temp+fire+temp*fire',data=forests).fit()
resultsF = modelF.summary()
#print(resultsF)
print(modelF.params)
#equations
## Full equation:
# FFMC = -8.1 +76.8*fire +2.4*temp-temp:fire*1.9
## For locations without fire:
# FFMC = -8.1 +76.8*0 +2.4*temp-temp:fire*0
#      = -8.1+2.4*temp
## For locations with fire:
# FFMC = -8.1 +76.8*1 +2.4*temp-temp:fire*1.9
#      = 68.7+0.5*temp

#interpretations
## For locations without fire:
# the intercept indicates that for location without fir, a temperature of zero degree Celsius is assosciate with a relative -8.1 fine fuel moisture code. 
## For locations with fire:
# the intercept indicates that for location with fir, a temperature of zero degree Celsius is assosciate with a relative 68.7 fine fuel moisture code. 

#plot regression lines
sns.lmplot(x='temp',y='FFMC',hue='fire',data=forests,
fit_reg=False)
plt.plot(forests.temp,
modelF.params[0]+modelF.params[1]*0+modelF.params[2]*forests.temp + modelF.params[3]*0,color='blue',linewidth=3, label='without fire')
plt.plot(forests.temp,
modelF.params[0]+modelF.params[1]*1+modelF.params[2]*forests.temp + modelF.params[3]*1*forests.temp,color='red',linewidth=3, label='with fire')
plt.legend()
plt.show()
plt.clf()

#plot FFMC vs humid
#sns.lmplot(x='humid',y='FFMC',data=forests,fit_reg=True)
sns.scatterplot(data=forests, x='humid',y='FFMC')
plt.show()
plt.clf()

#polynomial model predicting FFMC
resultsP= sm.OLS.from_formula('FFMC~ humid+np.square(humid)',data=forests).fit()
print(resultsP.params)
# another method: use np.power(___, ____)
#print(resultsP.params[0] + resultsP.params[1]*25 + resultsP.params[2]*np.power(25,2))

#regression equation
# FFMC = 116.6 - 0.75*humid - 0.01*humid^2

#sample predicted values
# 25%: FFMC = 116.6 - 0.75*25 - 0.01*25^2 = 91.6
# 35%: FFMC = 116.6 - 0.75*35 - 0.01*35^2 = 78.1
# 60%: FFMC = 116.6 - 0.75*60 - 0.01*60^2 = 35.6
# 70%: FFMC = 116.6 - 0.75*70 - 0.01*70^2 = 15.1
# I noticed that as the relative humidity level increase the FFMC decrease drastically. 

#interpretation of relationship
sns.lmplot(x='humid',y='FFMC',data=forests,fit_reg=False)
plt.plot(forests.humid, resultsP.params[0] + resultsP.params[1]*forests.humid + resultsP.params[2]*np.square(forests.humid), linewidth=2, color='red')
plt.show()
plt.clf()
# The relationship between FFMC and relative humidity is close to a polynomial regression line. A straight line can not accurately descripe their relationship. 

#multiple variables to predict FFMC
mult_model = sm.OLS.from_formula('FFMC~humid*temp*wind*rain', data=forests)
resultsMult = mult_model.fit()
print(resultsMult.params)
# include all variables and their interactions 

modelFFMC = sm.OLS.from_formula('FFMC ~ temp + rain + wind + humid',data=forests).fit()
print(modelFFMC.params)

#predict FWI from ISI and BUI
modelFWI = sm.OLS.from_formula('FWI~ISI+BUI', forests).fit()
print(modelFWI.params)
